In [1]:
import numpy as np
import pandas as pd
import lime
from lime.lime_text import LimeTextExplainer

In [2]:
df = pd.read_csv('data/IMDB Dataset.csv')

In [3]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


begin with train test split

In [4]:
from sklearn.model_selection import train_test_split

X = df.drop('sentiment', axis=1)
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizing

In [5]:
X_train = X_train.squeeze()
X_test = X_test.squeeze()

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

train the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train_vec, y_train)

model evaluation

In [ ]:
score = model.score(X_test_vec, y_test)
score

Create the explainer

In [ ]:
explainer = LimeTextExplainer(class_names=['negative', 'positive'], random_state = 42)

create a pipeline

In [ ]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(vectorizer, model)

select a sample

In [ ]:
idx = 80
sample_text = X_test.iloc[idx]
data_test = vectorizer.transform([sample_text])
prediction = model.predict(data_test)[0] 
y_true = np.array(y_test)[idx]   
print(f"Sample {idx} from the test set, predicted as {'positive' if prediction == 1 else 'negative'}, true label is {'positive' if y_true == 1 else 'negative'}")


In [ ]:
exp = explainer.explain_instance(
    X_test.iloc[idx],            
    pipeline.predict_proba,   
    num_features=6
)
exp.save_to_file('results/text_explanation1.html')
exp.show_in_notebook(text=sample_text)

Rerun the explainer again with a different seed

In [ ]:
explainer = LimeTextExplainer(class_names=['negative', 'positive'], random_state = 99)
exp2 = explainer.explain_instance(
    X_test.iloc[idx],            
    pipeline.predict_proba,   
    num_features=6
)
exp2.save_to_file('results/text_explanation2.html')
exp2.show_in_notebook(text=sample_text)

In [ ]:
sample_text = 'It is quite rare that a movie comes along that is so useless'
print("Original Text:", sample_text)

vectorizer = TfidfVectorizer()
model = RandomForestClassifier()

words = sample_text.split()

np.random.seed(42)
num_samples = 5
word_matrix = np.random.choice([0, 1], size=(num_samples, len(words)))
perturbed_sentences = []
probabilities = []

for row in word_matrix:
    perturbed_text = " ".join([word for word, keep in zip(words, row) if keep == 1])
    perturbed_sentences.append(perturbed_text)
    probabilities.append(np.random.uniform(0, 1)) 

weights = []
for row in word_matrix:
    proximity = 1 - (np.sum(row == 0) / len(words))
    weights.append(proximity)

columns = words + ["prob", "weight"]
data = np.hstack([word_matrix, np.array(probabilities).reshape(-1, 1), np.array(weights).reshape(-1, 1)])
df = pd.DataFrame(data, columns=columns)

In [ ]:

fig, ax = plt.subplots(figsize=(12, 2))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(df.columns))))
path = "results/simulated_text_perturbation.png.png"
fig.savefig(path, dpi=300, bbox_inches='tight')
plt.show()

